In [1]:
%pwd

'/home/jovyan/work/obstacle-detection/examples'

In [2]:
%cd ..

/home/jovyan/work/obstacle-detection


In [3]:
import sys
sys.path.append('/home/jovyan/work/boxes_vis/git/obstacle-detection/')
sys.path.append('/home/jovyan/work/obstacle-detection/model/')

In [4]:
sys.path

['/home/jovyan/work/obstacle-detection/examples',
 '/opt/conda/lib/python37.zip',
 '/opt/conda/lib/python3.7',
 '/opt/conda/lib/python3.7/lib-dynload',
 '',
 '/opt/conda/lib/python3.7/site-packages',
 '/opt/conda/lib/python3.7/site-packages/IPython/extensions',
 '/home/jovyan/.ipython',
 '/home/jovyan/work/boxes_vis/git/obstacle-detection/',
 '/home/jovyan/work/obstacle-detection/model/']

In [5]:
import yaml
import numpy as np
import pandas as pd
import pyntcloud


num ='000'
scan = np.fromfile('dataset/sequences/00/velodyne/000' + num + '.bin', dtype=np.float32)
scan = scan.reshape((-1, 4))

label = np.fromfile('dataset/sequences/00/labels/000' + num + '.label', dtype=np.uint32)
label = label.reshape((-1))

In [6]:
with open('config.yaml') as file:
    config = yaml.load(file, Loader=yaml.FullLoader)

In [7]:
obstacle_dct = config['obstacles']

In [8]:
# seg id -> seg name
obstacle_dct

{10: 'car',
 11: 'bicycle',
 13: 'bus',
 15: 'motorcycle',
 16: 'on-rails',
 18: 'truck',
 20: 'other-vehicle',
 30: 'person',
 31: 'bicyclist',
 32: 'motorcyclist',
 252: 'moving-car',
 253: 'moving-bicyclist',
 254: 'moving-person',
 255: 'moving-motorcyclist',
 256: 'moving-on-rails',
 257: 'moving-bus',
 258: 'moving-truck',
 259: 'moving-other-vehicle'}

In [9]:
leaning_map_obstacles = config['learning_map']

In [10]:
# ground truth id -> learning id
# leaning_map_obstacles

In [11]:
mapped_obstacles_lst = list(set([leaning_map_obstacles[id] for id in obstacle_dct.keys()]))
mapped_obstacles_lst

[1, 2, 3, 4, 5, 6, 7, 8]

In [12]:
import sys
sys.path.append("..")
from model.tasks.semantic.infer2 import get_user 

### Load Model

In [13]:
#user = get_user("dataset", "darknet21")
user = get_user("darknet21")

----------
INTERFACE:
model darknet21
----------

Opening arch config file from darknet21
Opening data config file from darknet21
model folder exists! Using model from darknet21
parser started
Using DarknetNet21 Backbone
Depth of backbone input =  5
Original OS:  32
New OS:  32
Strides:  [2, 2, 2, 2, 2]
Decoder original OS:  32
Decoder new OS:  32
Decoder strides:  [2, 2, 2, 2, 2]
Total number of parameters:  24712980
Total number of parameters requires_grad:  0
Param encoder  14921120
Param decoder  9786080
Param head  5780
Successfully loaded model backbone weights
Successfully loaded model decoder weights
Successfully loaded model head weights
Infering in device:  cpu


In [73]:
%timeit predicted_labels = user.infer('dataset/sequences/00/velodyne/000000.bin')

Infered scan  dataset/sequences/00/velodyne/000000.bin in 2.986173152923584 sec
Infered scan  dataset/sequences/00/velodyne/000000.bin in 3.015446186065674 sec
Infered scan  dataset/sequences/00/velodyne/000000.bin in 2.9522764682769775 sec
Infered scan  dataset/sequences/00/velodyne/000000.bin in 2.9182658195495605 sec
Infered scan  dataset/sequences/00/velodyne/000000.bin in 2.86429762840271 sec
Infered scan  dataset/sequences/00/velodyne/000000.bin in 2.9911413192749023 sec
Infered scan  dataset/sequences/00/velodyne/000000.bin in 3.4889438152313232 sec
Infered scan  dataset/sequences/00/velodyne/000000.bin in 3.6889333724975586 sec
3.21 s ± 311 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
predicted_labels = user.infer('dataset/sequences/00/velodyne/000000.bin')

Infered scan  dataset/sequences/00/velodyne/000000.bin in 3.0417513847351074 sec


In [15]:
predicted_labels.shape

(124668,)

### Visualize segmentated point cloud

In [ ]:
cloud = pd.DataFrame(scan[:, :3], columns=['x', 'y', 'z'])
cloud['seg_id'] = predicted_labels
pcloud = pyntcloud.PyntCloud(cloud)
pcloud.plot(use_as_color='seg_id', cmap="cool", backend="pythreejs")

![lol](img/darknet12_scan_000000_seq_00.png)

In [79]:
pcloud.points.seg_id.unique()

array([16, 15, 13, 19, 18, 14,  1,  8,  9, 10, 17, 11,  5,  7])

### Visualize only obstacles

In [ ]:
cloud_selected  = cloud[cloud.seg_id.isin(mapped_obstacles_lst)]
pcloud = pyntcloud.PyntCloud(cloud_selected)
pcloud.plot(use_as_color='seg_id', cmap="cool", backend="pythreejs")

![lol](img/darknet12_scan_000000_seq_00_obstacles.png)

### Load Model

In [20]:
#user = get_user("dataset", "darknet21")
user = get_user( "squeezeseg")

----------
INTERFACE:
model squeezeseg
----------

Opening arch config file from squeezeseg
Opening data config file from squeezeseg
model folder exists! Using model from squeezeseg
parser started
Using SqueezeNet Backbone
Depth of backbone input =  5
Original OS:  16
New OS:  16
Strides:  [2, 2, 2, 2]
Decoder original OS:  16
Decoder new OS:  16
Decoder strides:  [2, 2, 2, 2]
Total number of parameters:  915540
Total number of parameters requires_grad:  915540
Param encoder  724032
Param decoder  179968
Param head  11540
Successfully loaded model backbone weights
Successfully loaded model decoder weights
Successfully loaded model head weights
Infering in device:  cpu


In [21]:
%timeit predicted_labels = user.infer('dataset/sequences/00/velodyne/000000.bin')

Infered scan  dataset/sequences/00/velodyne/000000.bin in 0.6862523555755615 sec
Infered scan  dataset/sequences/00/velodyne/000000.bin in 0.6715078353881836 sec
Infered scan  dataset/sequences/00/velodyne/000000.bin in 0.6594150066375732 sec
Infered scan  dataset/sequences/00/velodyne/000000.bin in 0.6856420040130615 sec
Infered scan  dataset/sequences/00/velodyne/000000.bin in 0.6706273555755615 sec
Infered scan  dataset/sequences/00/velodyne/000000.bin in 0.752985954284668 sec
Infered scan  dataset/sequences/00/velodyne/000000.bin in 0.6574509143829346 sec
Infered scan  dataset/sequences/00/velodyne/000000.bin in 0.6702816486358643 sec
751 ms ± 29.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
predicted_labels = user.infer('dataset/sequences/00/velodyne/000000.bin')

Infered scan  dataset/sequences/00/velodyne/000000.bin in 0.653179407119751 sec


In [23]:
predicted_labels.shape

(124668,)

### Visualize segmentated point cloud

In [ ]:
cloud = pd.DataFrame(scan[:, :3], columns=['x', 'y', 'z'])
cloud['seg_id'] = predicted_labels
pcloud = pyntcloud.PyntCloud(cloud)
pcloud.plot(use_as_color='seg_id', cmap="cool", backend="pythreejs")

![lol](img/squeezeseg_scan_000000_seq_00.png)

In [25]:
pcloud.points.seg_id.unique()

array([18, 16, 19, 13, 15,  4,  5, 12,  1,  6,  8, 14, 17,  7, 10,  9, 11,
        3,  2])

### Visualize only obstacles

In [ ]:
cloud_selected  = cloud[cloud.seg_id.isin(mapped_obstacles_lst)]
pcloud = pyntcloud.PyntCloud(cloud_selected)
pcloud.plot(use_as_color='seg_id', cmap="cool", backend="pythreejs")

![lol](img/squeezeseg_scan_000000_seq_00_obstacles.png)